In [24]:
%%pyspark

from pyspark.sql.functions import lit

base_path = 'abfss://fssynwallodp01prod@sasynwallodp01prod.dfs.core.windows.net/'

subscriptions_prices_se_silver_path = base_path + 'Silver/Datastore/appxite_subscriptions_prices_country_se/'
subscriptions_prices_dk_silver_path = base_path + 'Silver/Datastore/appxite_subscriptions_prices_country_dk/'
subscriptions_prices_fi_silver_path = base_path + 'Silver/Datastore/appxite_subscriptions_prices_country_fi/'
subscriptions_prices_no_silver_path = base_path + 'Silver/Datastore/appxite_subscriptions_prices_country_no/'

subscriptions_details_se_silver_path = base_path + 'Silver/Datastore/appxite_subscriptions_details_country_se/'
subscriptions_details_dk_silver_path = base_path + 'Silver/Datastore/appxite_subscriptions_details_country_dk/'
subscriptions_details_fi_silver_path = base_path + 'Silver/Datastore/appxite_subscriptions_details_country_fi/'
subscriptions_details_no_silver_path = base_path + 'Silver/Datastore/appxite_subscriptions_details_country_no/'

subscriptions_se_silver_path = base_path + 'Silver/Datastore/appxite_subscriptions_country_se/'
subscriptions_dk_silver_path = base_path + 'Silver/Datastore/appxite_subscriptions_country_dk/'
subscriptions_fi_silver_path = base_path + 'Silver/Datastore/appxite_subscriptions_country_fi/'
subscriptions_no_silver_path = base_path + 'Silver/Datastore/appxite_subscriptions_country_no/'

customer_details_se_silver_path = base_path + 'Silver/Datastore/appxite_customer_details_country_se'
customer_details_dk_silver_path = base_path + 'Silver/Datastore/appxite_customer_details_country_dk'
customer_details_fi_silver_path = base_path + 'Silver/Datastore/appxite_customer_details_country_fi'
customer_details_no_silver_path = base_path + 'Silver/Datastore/appxite_customer_details_country_no'

cdm_account_gold = base_path + 'Gold/CDM/Account'

subscriptions_prices_columns = ["subscriptionId", "TotalCost"]

subscriptions_prices_se = (spark.read.load(subscriptions_prices_se_silver_path, format='delta').select(subscriptions_prices_columns).withColumn("dss_country", lit("se")))
subscriptions_prices_dk = (spark.read.load(subscriptions_prices_dk_silver_path, format='delta').select(subscriptions_prices_columns).withColumn("dss_country", lit("dk")))
subscriptions_prices_fi = (spark.read.load(subscriptions_prices_fi_silver_path, format='delta').select(subscriptions_prices_columns).withColumn("dss_country", lit("fi")))
subscriptions_prices_no = (spark.read.load(subscriptions_prices_no_silver_path, format='delta').select(subscriptions_prices_columns).withColumn("dss_country", lit("no")))

subscriptions_details_columns = ["Id", "AutoRenewalDetails", "StartDate"]

subscriptions_details_se = (spark.read.load(subscriptions_details_se_silver_path, format='delta').select(subscriptions_details_columns).withColumn("dss_country", lit("se")))
subscriptions_details_dk = (spark.read.load(subscriptions_details_dk_silver_path, format='delta').select(subscriptions_details_columns).withColumn("dss_country", lit("dk")))
subscriptions_details_fi = (spark.read.load(subscriptions_details_fi_silver_path, format='delta').select(subscriptions_details_columns).withColumn("dss_country", lit("fi")))
subscriptions_details_no = (spark.read.load(subscriptions_details_no_silver_path, format='delta').select(subscriptions_details_columns).withColumn("dss_country", lit("no")))

subscriptions_columns = ["Id", "Status", "Name", "Vendor", "Quantity", "ExpirationTimestamp","x_on_behalf_of"]

subscriptions_se = (spark.read.load(subscriptions_se_silver_path, format='delta').select(subscriptions_columns).withColumn("dss_country", lit("se")))
subscriptions_dk = (spark.read.load(subscriptions_dk_silver_path, format='delta').select(subscriptions_columns).withColumn("dss_country", lit("dk")))
subscriptions_fi = (spark.read.load(subscriptions_fi_silver_path, format='delta').select(subscriptions_columns).withColumn("dss_country", lit("fi")))
subscriptions_no = (spark.read.load(subscriptions_no_silver_path, format='delta').select(subscriptions_columns).withColumn("dss_country", lit("no")))

customer_details_columns = ["Id", "ErpId", "VatNumber", "RegistrationNumber", "PartnerName"]

customer_details_se = (spark.read.load(customer_details_se_silver_path, format='delta').select(customer_details_columns).withColumn("dss_country", lit("se")))
customer_details_dk = (spark.read.load(customer_details_dk_silver_path, format='delta').select(customer_details_columns).withColumn("dss_country", lit("dk")))
customer_details_fi = (spark.read.load(customer_details_fi_silver_path, format='delta').select(customer_details_columns).withColumn("dss_country", lit("fi")))
customer_details_no = (spark.read.load(customer_details_no_silver_path, format='delta').select(customer_details_columns).withColumn("dss_country", lit("no")))

cdm_account_columns = ["accountId","ErpId","VatNumber","RegistrationNumber","PartnerName"]

# take only accountNumber from Account table is in the cdm : accountNumber = iifNull(toString(ErpId),VatNumber, toString(RegistrationNumber))
cdm_account = spark.read.load(cdm_account_gold, format='delta').select(cdm_account_columns)

In [2]:
%%pyspark
display(cdm_account.limit(5))
cdm_account.printSchema()

In [92]:
%%pyspark

from pyspark.sql import functions as F

# Combine all four DataFrames into a single DataFrame
combined_subscriptions_prices = (subscriptions_prices_se
                                 .union(subscriptions_prices_dk)
                                 .union(subscriptions_prices_fi)
                                 .union(subscriptions_prices_no))

# Rename the "subscriptionId" column to "Id" in combined_subscriptions_prices DataFrame
combined_subscriptions_prices = combined_subscriptions_prices.withColumnRenamed("subscriptionId", "Id")

# Count the total number of rows in the combined DataFrame
total_rows = combined_subscriptions_prices.count()
print(f"Total number of rows combined_subscriptions_prices: {total_rows}")

# Count the number of distinct rows based on all columns
distinct_rows = combined_subscriptions_prices.distinct().count()
print(f"Total number of distinct rows combined_subscriptions_prices: {distinct_rows}")

display(combined_subscriptions_prices.limit(5))
combined_subscriptions_prices.printSchema()

In [25]:
%%pyspark

# Combine all four DataFrames into a single DataFrame
subscriptions_details_combined = (subscriptions_details_se
                                 .union(subscriptions_details_dk)
                                 .union(subscriptions_details_fi)
                                 .union(subscriptions_details_no))

# Count the total number of rows in the combined DataFrame
total_rows = subscriptions_details_combined.count()
print(f"Total Rows: {total_rows}")

# Count the number of distinct rows based on all columns
distinct_rows = subscriptions_details_combined.distinct().count()
print(f"Distinct Rows: {distinct_rows}")

display(subscriptions_details_combined.limit(5))
subscriptions_details_combined.printSchema()

In [26]:
%%pyspark

delta_table_path = 'abfss://fssynwallodp01prod@sasynwallodp01prod.dfs.core.windows.net/Gold/test/SubscriptionDetailsCombined/'

subscriptions_details_combined.write.mode("overwrite").option("overwriteSchema", "true").format('delta').save(delta_table_path)

In [107]:
%%pyspark

# Combine all four DataFrames into a single DataFrame
combined_subscriptions = (subscriptions_se
                         .union(subscriptions_dk)
                         .union(subscriptions_fi)
                         .union(subscriptions_no))

# Count the total number of rows in the combined DataFrame
total_rows = combined_subscriptions.count()
print(f"Total Rows: {total_rows}")

# Count the number of distinct rows based on all columns
distinct_rows = combined_subscriptions.distinct().count()
print(f"Distinct Rows: {distinct_rows}")

In [86]:
%%pyspark
# Combine all four DataFrames into a single DataFrame
combined_customer_details = (customer_details_se
                           .union(customer_details_dk)
                           .union(customer_details_fi)
                           .union(customer_details_no))

# Count the total number of rows in the combined DataFrame
total_rows = combined_customer_details.count()
print(f"Total Rows: {total_rows}")

# Count the number of distinct rows based on all columns
distinct_rows = combined_customer_details.distinct().count()
print(f"Distinct Rows: {distinct_rows}")

In [93]:
%%pyspark
combined_subscriptions.printSchema()
subscriptions_details_combined.printSchema()
combined_subscriptions_prices.printSchema()

In [115]:
%%pyspark

from pyspark.sql.functions import col

# Perform inner joins
joined_df = subscriptions_details_combined \
    .join(combined_subscriptions, subscriptions_details_combined["Id"] == combined_subscriptions["Id"], "full") \
    .join(combined_subscriptions_prices, combined_subscriptions["Id"] == combined_subscriptions_prices["Id"], "full") \
    .select(
        subscriptions_details_combined["Id"].alias("subscriptionId"),
        subscriptions_details_combined["AutoRenewalDetails"],
        subscriptions_details_combined["StartDate"],
        combined_subscriptions["Status"],
        combined_subscriptions["Name"],
        combined_subscriptions["Vendor"],
        combined_subscriptions["Quantity"],
        combined_subscriptions["ExpirationTimestamp"],
        combined_subscriptions["x_on_behalf_of"],
        combined_subscriptions_prices["TotalCost"]
    )

# Show the joined DataFrame
display(joined_df.limit(5))
joined_df.printSchema()
print(f"Joined DataFrame has {joined_df.count()} rows.")

In [117]:
%%pyspark
from pyspark.sql.functions import col

# Filter rows where "Id" is not empty or null
filtered_df = joined_df.filter(col("subscriptionId").isNotNull() & (col("subscriptionId") != ""))

# Show the filtered DataFrame
display(filtered_df.limit(5))
filtered_df.printSchema()
print(f"Filtered DataFrame has {filtered_df.count()} rows.")

In [100]:
%%pyspark

from pyspark.sql.functions import col, lit, when

# Use when and otherwise to create the accountNumber column
cdm_account_accountNumber = cdm_account.withColumn(
    "accountNumber",
    when(col("ErpId").isNotNull(), col("ErpId").cast("string"))
    .otherwise(when(col("VatNumber").isNotNull(), col("VatNumber").cast("string"))
    .otherwise(col("RegistrationNumber").cast("string")))
)

# Show the result DataFrame
cdm_account_accountNumber.printSchema()

In [104]:
%%pyspark

from pyspark.sql.functions import col, lit, when

# Use when and otherwise to create the accountNumber column
combined_customer_details_accountNumber = combined_customer_details.withColumn(
    "accountNumber",
    when(col("ErpId").isNotNull(), col("ErpId").cast("string"))
    .otherwise(when(col("VatNumber").isNotNull(), col("VatNumber").cast("string"))
    .otherwise(col("RegistrationNumber").cast("string")))
)

# Show the result DataFrame
combined_customer_details_accountNumber.printSchema()

In [118]:
%%pyspark
combined_customer_details_accountNumber.printSchema()
cdm_account_accountNumber.printSchema()
filtered_df.printSchema()

In [120]:
# Perform an inner join on the "PK x_on_behalf_of" column in filtered_df and "PK Id" column in combined_customer_details_accountNumber
subscription_customer_details_joined = filtered_df.join(
    combined_customer_details_accountNumber,
    filtered_df["x_on_behalf_of"] == combined_customer_details_accountNumber["Id"],
    how="inner"
)

# Show the joined DataFrame
display(subscription_customer_details_joined.limit(5))
subscription_customer_details_joined.printSchema()
print(f"Joined DataFrame has {subscription_customer_details_joined.count()} rows.")


In [122]:
%%pyspark
# Perform an inner join based on "accountNumber" and "PartnerName"
final_df = subscription_customer_details_joined.join(
    cdm_account_accountNumber,
    on=["accountNumber", "PartnerName"],
    how="inner"
)

# Show the joined DataFrame
display(final_df.limit(5))
final_df.printSchema()
print(f"Joined DataFrame has {final_df.count()} rows.")

In [123]:
%%pyspark
from pyspark.sql.functions import col, when, to_date, expr
from pyspark.sql.types import DecimalType

# Select and transform columns
transformed_df = final_df.select(
    col("subscriptionId"),
    col("AutoRenewalDetails"),
    to_date(col("StartDate")).alias("StartDate"),  # Transform StartDate to date
    when(col("Status") == "Enabled", 1).otherwise(0).alias("Active"),  # Transform Status to Active column
    col("Name"),
    col("Vendor").alias("VendorName"),  # Rename Vendor to VendorName
    col("Quantity").cast(DecimalType(10, 2)).alias("Quantity"),  # Transform Quantity to decimal
    to_date(col("ExpirationTimestamp")).alias("LatestEndDate"),  # Transform ExpirationTimestamp to date
    col("TotalCost").cast(DecimalType(10, 2)).alias("TotalPrice"),  # Transform TotalCost to decimal
    col("accountId")
)

# Show the transformed DataFrame
display(transformed_df.limit(5))
transformed_df.printSchema()


In [20]:
%%pyspark
base_path = 'abfss://fssynwallodp01prod@sasynwallodp01prod.dfs.core.windows.net/'
subscription_path = base_path + 'Gold/CDM/Subscription/'

subscription = spark.read.load(subscription_path, format='delta')

display(subscription.limit(5))
subscription.printSchema()

In [21]:
%%pyspark
subscription_renamed = subscription.withColumnRenamed("Active", "Status")
subscription_renamed.printSchema()

In [8]:
%%pyspark
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, TimestampType
from datetime import datetime

# Define the values you want to add
current_date = datetime.now()

dss_load_time = current_date
dss_create_time = current_date
dss_update_time = current_date
dss_country = "country"
dss_record_source = "df_cdm_subscription_attributes"

# Add the columns and values to the DataFrame
subscription_df = subscription \
    .withColumn("dss_load_time", F.lit(dss_load_time).cast(TimestampType())) \
    .withColumn("dss_create_time", F.lit(dss_create_time).cast(TimestampType())) \
    .withColumn("dss_update_time", F.lit(dss_update_time).cast(TimestampType())) \
    .withColumn("dss_record_source", F.lit(dss_record_source).cast(StringType())) \
    .withColumn("dss_country", F.lit(dss_country).cast(StringType()))

subscription_df.printSchema()

In [9]:
display(subscription_df.limit(5))

In [22]:
%%pyspark

delta_table_path = 'abfss://fssynwallodp01prod@sasynwallodp01prod.dfs.core.windows.net/Gold/CDM/Subscription/'

subscription_renamed.write.mode("overwrite").option("overwriteSchema", "true").format('delta').save(delta_table_path)

In [11]:
%%pyspark
from pyspark.sql.functions import countDistinct, col
from datetime import datetime

# Current date
current_date = datetime.now().strftime('%d-%m-%Y')

subscription_gold_path= 'Gold/CDM/Subscription/'

# Print current date and silver_se_path
print(f"Current Date: {current_date}")
print(f"Order In Sales Path: {subscription_gold_path}\n")

# Count the total number of rows
total_rows = subscription.count()

# Count the number of distinct 'Id' rows
distinct_id_rows = subscription.select(countDistinct('subscriptionId')).first()[0]

# Filter rows where 'Id' is empty
empty_id_rows_df = subscription.filter(col('subscriptionId').isNull())

# Count rows where 'Id' is empty
id_rows = empty_id_rows_df.count()

# Display the comparison
print(f"Total Rows: {total_rows}")
print(f"Distinct 'Id' Rows: {distinct_id_rows}")
print(f"Id Rows empty: {id_rows}\n")

# Display rows with empty 'Id'
# display(empty_id_rows_df)
